# REWOO-ReAct Hybrid Pattern Implementation with Strands Multiagent Graph

## Overview

This implementation demonstrates a **hybrid REWOO-ReAct pattern** using the Strands multiagent graph framework. It combines REWOO's structured planning approach with ReAct's direct tool execution, creating a two-stage system that balances planning reliability with execution efficiency.

## Architecture

The system consists of two specialized agents connected in a sequential graph:

```
User Query → [Planner] → [Solver] → Final Response
```

### 1. Planner Agent (REWOO-style)
- **Purpose**: Generates structured execution plans without executing tools
- **Input**: User query (e.g., "Check my flight and change to earlier time")
- **Output**: Structured plan with numbered steps:
  ```
  Plan 1: Retrieve flight information
  #E1 = get_reservation_details[reservation_id="3442587242"]
  
  Plan 2: Search for earlier flights
  #E2 = search_direct_flight[origin=origin_code, destination=dest_code, date=travel_date]
  
  Plan 3: Update reservation with selected flight
  #E3 = update_reservation_flights[reservation_id="3442587242", cabin=cabin_class, flights=selected_flights, payment_id=payment_info]
  ```
- **Implementation**: Custom agent class using `generate_flight_plan` tool

### 2. Solver Agent (ReAct-style)
- **Purpose**: Executes the plan using ReAct pattern with direct tool access
- **Input**: Structured plan from planner + original user query
- **Process**: 
  - Receives the plan as guidance for tool execution order
  - Uses ReAct reasoning to execute tools with full access to airline tools
  - Can adapt execution based on intermediate results
  - Provides natural language responses
- **Output**: Final user response with completed actions
- **Implementation**: ReAct agent with `solve_flight_query` tool and full tool suite

## Key Technical Details

### Hybrid Approach Benefits
- **Planning Reliability**: REWOO-style planner ensures systematic approach
- **Execution Flexibility**: ReAct-style solver can adapt during execution
- **Reduced Latency**: Only 2 agents vs 3 in pure REWOO
- **Better Error Handling**: ReAct solver can recover from plan deviations

### Custom Agent Classes
Due to multiagent graph requirements, each agent extends the base `Agent` class with custom `stream_async()` methods:

```python
class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        message = Message(content=[{"text": str(plan_result)}])
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=EventLoopMetrics(),
            state=None
        )
        yield {"result": agent_result}

class SolverAgent(Agent):
    async def stream_async(self, prompt: str):
        parsed = extract_original_task_and_plan(prompt)
        final_answer = self.tool.solve_flight_query(
            user_query=parsed["original_task"],
            plan=parsed["plan_text"]
        )
        # ... yield AgentResult
```

### Information Flow
1. **Planner Input**: Raw user query
2. **Solver Input**: Multiagent graph automatically combines:
   - Original task
   - Planner output (the structured plan)
3. **Solver Processing**: Uses plan as guidance while maintaining ReAct flexibility

## Usage

```python
# Create the hybrid multiagent graph
rewoo_react_graph = create_rewoo_react_graph()

# Execute with user query
result = rewoo_react_graph(
    "Hi what time is my flight, my passenger id is '3442 587242'. Change my flight to earlier time."
)

# Access results from each agent
planner_result = result.results['planner']  # The structured plan
solver_result = result.results['solver']    # Final executed response
```

This hybrid implementation showcases how the Strands multiagent graph framework can combine different agent patterns to create more efficient and reliable workflows that leverage the strengths of both REWOO planning and ReAct execution.

In [ ]:
!pip3 install -r ./requirements.txt --quiet --upgrade
!pip3 install strands-agents strands-agents-tools --quiet


In [ ]:
import time
import boto3
import ipywidgets as widgets
import uuid
import pandas as pd
import numpy as np
import os
import shutil
import sqlite3
import functools
import requests
import pytz
import warnings
from IPython.display import Image, display
from botocore.config import Config
from typing import Annotated, Literal, Optional, Union
from typing_extensions import TypedDict
from bs4 import BeautifulSoup
from datetime import date, datetime
from typing import List, Dict, Any
import re
import json
import base64


from strands import Agent
from strands import tool
from strands.models import BedrockModel
from strands.agent.conversation_manager import SlidingWindowConversationManager

from strands.multiagent.graph import GraphBuilder
from strands.agent import AgentResult
from strands.types.content import Message
from strands.types.streaming import StopReason
from strands.telemetry.metrics import EventLoopMetrics
from strands.telemetry.config import StrandsTelemetry
import logging

from rewoo_react_helper_funcs import *
from bedrock_helper import get_bedrock_response, get_claude_response, get_claude_response_text

### Use Strands 


In [ ]:
#Clients

region = "us-east-1"

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)



# Create BedrockModel with specified region
bedrock_model_taubench = BedrockModel(region_name= region)

#Conversation Manager
conv_manager = SlidingWindowConversationManager(window_size=10)


#setup logging
# Disable all logging except critical errors
logging.basicConfig(level=logging.CRITICAL)

# Silence specific noisy loggers completely
for logger_name in ["strands", "graph", "event_loop", "registry", "sliding_window_conversation_manager", "bedrock", "streaming"]:
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)


## Get MAbench and Taubench tools

In [ ]:
# Libraries

import sys
sys.path.append('../data/ma-bench/')
sys.path.append('../data/tau-bench/')

from mabench.environments.airline.tools.book_reservation import book_reservation
from mabench.environments.airline.tools.calculate import calculate
from mabench.environments.airline.tools.cancel_reservation import cancel_reservation
from mabench.environments.airline.tools.get_reservation_details import get_reservation_details
from mabench.environments.airline.tools.get_user_details import get_user_details
from mabench.environments.airline.tools.list_all_airports import list_all_airports
from mabench.environments.airline.tools.search_direct_flight import search_direct_flight
from mabench.environments.airline.tools.search_onestop_flight import search_onestop_flight
from mabench.environments.airline.tools.send_certificate import send_certificate
from mabench.environments.airline.tools.think import think
from mabench.environments.airline.tools.transfer_to_human_agents import transfer_to_human_agents
from mabench.environments.airline.tools.update_reservation_baggages import update_reservation_baggages
from mabench.environments.airline.tools.update_reservation_flights import update_reservation_flights
from mabench.environments.airline.tools.update_reservation_passengers import update_reservation_passengers

domain = "airline"

# from tau_bench.envs.tool import Tool
# from tau_bench.envs.airline.tools import *
from tau_bench.envs.airline.data import *
from tau_bench.envs.airline.tasks import *
from tau_bench.envs.airline.wiki import WIKI

### REWOO Orchestration

### PLANNER : Receives user query and makes the plan
"""
PLANNER AGENT - REWOO Step 1: Reasoning
Generates structured execution plans without tool execution. Takes user queries and creates 
step-by-step plans in #E1, #E2, #E3 format specifying which tools to call and with what parameters.
Custom stream_async() ensures proper AgentResult yielding for multiagent graph compatibility. (in last cell of notebook)
"""

In [ ]:
def direct_llm_call(prompt):
    max_tokens = 2048
    temp = 0
    topP = 1
    response = get_claude_response(user_message=prompt,
                                    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0", 
                                    max_tokens=max_tokens, 
                                   temp=temp)                
    answer = get_claude_response_text(response)
    return answer


In [ ]:
@tool
def generate_flight_plan(user_query: str) -> str:
    """Generate a structured flight plan for the given user query"""

    print(f"\n INSIDE generate_flight_plan tool \n")
    # planner prompt
    planner_prompt = """
# PLANNING ONLY ASSISTANT - DO NOT EXECUTE

Your ONLY job is to write a plan using the exact format below. You MUST NOT try to execute the plan or have any other interactions.

## Available Flight Tools
* calculate[expression]
* get_reservation_details[reservation_id]
* update_reservation_flights[reservation_id, cabin, flights, payment_id]
* search_onestop_flight[origin, destination, date]
* send_certificate[user_id, amount]
* cancel_reservation[reservation_id]
* search_direct_flight[origin, destination, date]
* get_user_details[user_id]
* list_all_airports[]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* think[thought]
* transfer_to_human_agents[summary]
* update_reservation_passengers[reservation_id, passengers]
* update_reservation_baggages[reservation_id, total_baggages, nonfree_baggages, payment_id]
* book_reservation[user_id, origin, destination, flight_type, cabin, flights, passengers, payment_methods, total_baggages, nonfree_baggages, insurance]
* cancel_reservation[reservation_id]
* calculate[expression]

## REPEAT Syntax and Usage
When multiple iterations of the same steps are needed, use this format:

1. First, use think tool to analyze and count items to process
2. Then, use another think tool to plan iteration details
3. Finally, use REPEAT block with the count from previous steps

REPEAT(count_from_previous_step) {
    tool1[parameters]
    tool2[parameters]
    ...
}

Available variables in REPEAT blocks:
- CURRENT_ITERATION (0-based index)
- CURRENT_ITEM (from list being processed)
- Other variables extracted from previous steps

Use REPEAT blocks when:
- Processing multiple reservations
- Applying multiple certificates
- Handling multiple passengers
- Any task that requires the same steps multiple times

Note: Evidence numbers inside REPEAT will be expanded sequentially


## Required Format - USE EXACTLY THIS:

Plan 1: [Description]
#E1 = [tool_name][parameters]

Plan 2: [Description]
#E2 = [tool_name][parameters]

## Examples:


Example 1 : "Can you put me on an earlier flight? My reservation ID is 'CD789012'"
Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="CD789012"]

Plan 2: Search for earlier direct flights based on the origin, destination and date from #E1
#E2 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 3: Update the reservation with the earlier flight found in #E2 and use details from #E1 and useer query as necessary
#E3 = update_reservation_flights[reservation_id="CD789012", cabin=cabin_class, flights=selected_flights, payment_id=payment_info]

Example 2 : "My user id is mia_li_3668. I want to fly from New York to Seattle on May 20 (one way). I do not want to fly before 11am EST. I want to fly in economy. I prefer direct flights but one stopover is also fine. If there are multiple options, I prefer the one with the lowest price. I have 3 baggages. I do not want insurance. I want to use my two certificates to pay. If only one certificate can be used, I prefer using the larger one, and pay the rest with my 7447 card"
Plan 1: Get user details to check available certificates
#E1 = get_user_details[user_id="mia_li_3668"]

Plan 2: Get list of airports to find the airport codes for New York and Seattle
#E2 = list_all_airports[]

Plan 3: Search for direct flights using airport codes from #E2 and date from given user question
#E3 = search_direct_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 4: If no suitable direct flights, search for one-stop flights using using airport codes from #E2 and date from given user question
#E4 = search_onestop_flight[origin=origin_airport_code, destination=destination_airport_code, date=travel_date]

Plan 5: Return selected flights from #E4  and #E3 


Example 3 : "I have a booking number TR7845. I need to update my daughter's name from Emma Wilson to Emma Thompson as she recently got married. I'm Jennifer Wilson, ID: TW5432P891."

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="TR7845"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="TW5432P891"]

Plan 3: Think about the passenger name changes needed
#E3 = think["Analyze the user query and reservation details:

Identify the passenger whose name needs to be changed: Emma Wilson
New name for this passenger: Emma Thompson
Keep all other passengers unchanged
Preserve existing passenger details (DOB, etc) from reservation
Create an updated passenger list with the name change"]

Plan 4: Update the reservation with modified passenger information
#E4 = update_reservation_passengers[reservation_id="TR7845", passengers=[
{"first_name": "Jennifer", "last_name": "Wilson", "dob": extracted_dob_jennifer},
{"first_name": "Emma", "last_name": "Thompson", "dob": extracted_dob_emma}
]]

Example 4 : "Hi, my name is Jordan Smith (customer ID: ZX7890Y123). I have a reservation with booking code LM5678 for a flight from Chicago to Miami on June 15. 
I need to add my son, Alex Smith, to the reservation and include an extra bag for him. Can you help me with that?"

Plan 1: Retrieve the current reservation details
#E1 = get_reservation_details[reservation_id="LM5678"]

Plan 2: Verify user identity and authorization
#E2 = get_user_details[user_id="ZX7890Y123"]

Plan 3: Think about required passenger updates
#E3 = think["Analyze current reservation and requested changes:

Get existing passenger list from #E1
New passenger to add: Alex Smith (son)
Need to preserve all existing passenger details
Create updated passenger list that includes both existing and new passengers"]

Plan 4: Update the reservation with complete passenger list
#E4 = update_reservation_passengers[reservation_id="LM5678", passengers=[
extract_existing_passengers_from_E1,
{"first_name": "Alex", "last_name": "Smith", "type": "child"}
]]

Plan 5: Think about baggage update
#E5 = think["Calculate baggage updates:

Get current total_baggages from #E1
Add one extra bag for new passenger
Determine if extra bag is free or paid based on cabin class"]

Plan 6: Update the baggage count
#E6 = update_reservation_baggages[
reservation_id="LM5678",
total_baggages=current_total_plus_one,
nonfree_baggages=current_nonfree_plus_one,
payment_id=payment_from_context
]

Example 5: "My user id is ABC123. I want to downgrade all my business flights to economy for my reservations. Please calculate total savings."

Plan 1: Get user details to retrieve all reservations
#E1 = get_user_details[user_id="ABC123"]

Plan 2: REPEAT(length_of_reservations_from_#E1) {
    get_reservation_details[reservation_id=CURRENT_RESERVATION_ID]    
    calculate["current_savings = business_fare - economy_fare"]
    calculate["total_savings += current_savings"]
    update_reservation_flights[reservation_id=CURRENT_RESERVATION_ID, cabin="economy", flights=CURRENT_FLIGHTS, payment_id=CURRENT_PAYMENT]
}


## IMPORTANT: 
1. ONLY write the plan - nothing else
2. Do NOT add any explanations or clarifications
3. Do NOT attempt to execute any actions
4. Follow the format exactly as shown
5. Use 'think' tool only when needed like name change.


<policy>
{policy}
</policy>
"""
    
    planning_llm = Agent(
        model=bedrock_model_taubench,
        system_prompt=planner_prompt.replace("{policy}", WIKI)
    )
    plan = planning_llm(user_query)
    
    return str(plan)



### SOLVER: Receives full plan and the responses of individual tool calls and prepares final response which is given to the user
"""
SOLVER AGENT - REWOO Step 3: Synthesis  
Combines execution evidence from Worker with original user query to generate final response.
Receives structured evidence dictionary and synthesizes it into natural language answer.
Uses solve_flight_query tool to create user-friendly responses from technical execution results.
"""

In [ ]:

tools=[book_reservation,
        calculate,
        cancel_reservation,
        get_reservation_details,
        get_user_details,
        list_all_airports,
        search_direct_flight,
        search_onestop_flight,
        send_certificate,
        think,
        transfer_to_human_agents,
        update_reservation_baggages,
        update_reservation_flights,
        update_reservation_passengers
      ],


solve_prompt = """# Operating Mode (Plan-Guided ReAct)
- You must execute ONLY the tools and steps authorized in <plan>, in order (#E1..#En). Do not invent new steps or tools.
- Within each step you may think→act→observe (ReAct), but:
  - Use concrete, non-placeholder arguments only.
  - Resolve cross-step references from prior results (#E*), the user task, and the policy.
  - Validate argument types/enums; infer airport states (e.g., “Houston → Texas”) and ensure city↔state consistency.
  - If a step cannot run (policy or missing data), STOP that branch and produce a policy-aligned outcome (e.g., suggest cancel→rebook for Basic Economy).

# Tool & Policy Discipline
- Never call tools not listed in <plan>. Never change the step order.
- Before any mutating call (book/update/cancel/refund/charge):
  - Check fare class, change/refund windows, baggage/insurance rules, payment priorities, and user authorizations.
  - If the user request is **clear and in-policy**, proceed without redundant confirmation.
  - If ambiguity or irreversible risk exists, ask exactly one targeted clarification before acting.
- Do not fabricate IDs, flight numbers, prices, or payment methods.
- Keep side effects idempotent: avoid double updates/charges; summarize deltas precisely (what changed, totals, sources of funds).

# Output Contract
- Return only what the user needs: the answer and/or a concise confirmation with key details (itinerary deltas, amounts, instruments used).
- You must reveal internal thoughts, tool traces, or step logs unless explicitly asked.

# Execution Template 
for step in plan(#E1..#En):
  think (in <think> tags) → call tool with validated args → observe → update working memory
after last step:
  synthesize final user-facing answer that reflects policy, tool outputs, and the task


<policy>
{policy}
</policy>

<plan>
{plan}
</plan>

Response:
"""


@tool
def solve_flight_query(plan: str, user_query: str) -> str:
    """
    Solve user query using structured evidence from worker execution
    """

    try:

        
        formatted_prompt = solve_prompt.format(plan=plan, policy=WIKI)
        agent = Agent(tools=tools, system_prompt=formatted_prompt)
        print(f" *********** starting rewoo guided react response ************* \n" )
        react_response = agent(user_query)   
        
        result = str(react_response)
        return result
        
    except Exception as e:
        print(f"DEBUG: Exception in solve_flight_query: {e}")
        print(f"DEBUG: Exception type: {type(e)}")
        raise

## Make the REWOO strands graph

In [ ]:
# Build rewoo graph
class PlannerAgent(Agent):
    async def stream_async(self, prompt: str):
        # Call the tool and get result
        print(f"DEBUG: PLANNER AGENT CALLED \n")
        prompt=normalize_prompt(prompt)
        plan_result = self.tool.generate_flight_plan(user_query=prompt)
        
        # Create  AgentResult object with required parameters
        message = Message(content=[{"text": str(plan_result)}])
        metrics = EventLoopMetrics()
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        # Yield the result event that multiagent graph expects
        yield {"result": agent_result}

# Use Custom planner
planner_agent = PlannerAgent(
    model=bedrock_model_taubench,
    tools=[generate_flight_plan],
    name="planner"
)


class SolverAgent(Agent):
    async def stream_async(self, prompt: str):
        # Extract plan and evidence from the combined input
        # The prompt will contain both original task and worker results
        print(f"DEBUG: SOLVER AGENT CALLED TO FORM FINAL ANSWER FROM EXECUTED PLAN\n")
        prompt=normalize_prompt(prompt)
        parsed = extract_original_task_and_plan(prompt)
        print("ORIGINAL TASK:\n", parsed["original_task"], "\n")
        print("PLAN TEXT:\n", parsed["plan_text"], "\n")
        # Call solve_flight_query tool
        final_answer = self.tool.solve_flight_query(
            user_query=parsed["original_task"],
            plan=parsed["plan_text"]
            
        )
        
        # Create AgentResult object
        message = Message(content=[{"text": str(final_answer)}])
        metrics = EventLoopMetrics() # check how to get the eventloopmetrics
        
        agent_result = AgentResult(
            stop_reason="end_turn",
            message=message,
            metrics=metrics,
            state=None
        )
        
        yield {"result": agent_result}


# Use the custom solver
solver_agent = SolverAgent(
    model=bedrock_model_taubench,
    tools=[solve_flight_query],
    name="solver"
)


# Finally create the graph with the 2 agent nodes
def create_rewoo_react_graph():

   
    builder = GraphBuilder()
    
    # Add the three agents
    builder.add_node(planner_agent, "planner")    
    builder.add_node(solver_agent, "solver")
    
    # Sequential flow: planner  -> solver
    builder.add_edge("planner", "solver")
    
    builder.set_entry_point("planner")
    return builder.build()



In [ ]:
### Load the taubench dataset

In [ ]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)


In [ ]:
# Create and execute
import json
import ast
import time

def extract_text_from_response(response_str):
    try:
        # First try to parse as JSON
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        try:
            # If JSON fails, try ast.literal_eval
            response_dict = ast.literal_eval(response_str)
        except:
            return "Error parsing response"
    
    # Extract text from content
    try:
        return response_dict['content'][0]['text']
    except (KeyError, IndexError):
        return "Error extracting text"
        

# previous without getting metrics
def test_rewoo_react_graph(question_id):
    task = tasks[question_id]
    user_query = task["question"]
    user_id = task['user_id']
    session_id= uuid.uuid4()
    rewoo_react_graph = create_rewoo_react_graph()
    #rewoo_graph = create_rewoo_graph(user_id, session_id, question_id)
    start=time.time()
    result = rewoo_react_graph(user_query)
    
    exec_time=time.time()-start
    print("=== REWOO Multiagent Graph Results ===")
    print(f"Graph execution time: {exec_time}")
    #print(f"Metrics: {result.metrics}")
    print(f"Status: {result.status}")
    print(f"Total nodes: {result.total_nodes}")
    print(f"Completed nodes: {result.completed_nodes}")
    filename = f"./output/rewoo_react_response_{question_id}.txt"
    
    try:
       
        with open(filename, "w", encoding="utf-8") as f:
            # Write execution summary
            f.write("=== REWOO Multiagent Graph Results ===\n")
            f.write(f"Status: {result.status}\n")
            f.write(f"Total nodes: {result.total_nodes}\n")
            f.write(f"Completed nodes: {result.completed_nodes}\n\n")
          
            # Write each node's result
            for node_id, node_result in result.results.items():
                print(f"\n--- {node_id.upper()} ---")
                
                # Write node separator
                f.write(f"\n{'='*50}\n")
                f.write(f"--- {node_id.upper()} ---\n")
                f.write(f"{'='*50}\n")
                
                try:
                    if hasattr(node_result.result, 'content'):
                        content = node_result.result.content
                        print(content)
                        f.write(str(content) + "\n")
                    else:
                        result_text = extract_text_from_response(str(node_result.result))
                        print(result_text)
                        f.write(result_text + "\n")
                except Exception as e:
                    error_msg = f"Error processing {node_id}: {str(e)}"
                    print(error_msg)
                    f.write(error_msg + "\n")
                
                f.write("\n")  # Add blank line between nodes
                
    except Exception as e:
        print(f"Error writing to file {filename}: {str(e)}")
    

### Test with question_id

In [ ]:
question_id = 8 #20, #48

test_rewoo_graph(question_id)

